In [1]:
# link: https://github.com/liyaguang/DCRNN
import numpy as np
import pandas as pd
import json

In [2]:
import os
import pandas as pd
import geopandas as gpd
import tqdm
import datetime
import numpy as np
import matplotlib.pyplot as plt
from cartoframes.viz import *
from tqdm import notebook

In [3]:
def ensure_dir(dir_path):
    """Make sure the directory exists, if it does not exist, create it.

    Args:
        dir_path (str): directory path
    """
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [4]:
import os
import datetime
# link: https://github.com/liyaguang/DCRNN
import numpy as np
import pandas as pd
import json

dataset_name = 'SSUB'
outputdir = f'../Bigscity-LibCity/raw_data/{dataset_name}'
ensure_dir(outputdir)
dataname = outputdir+f'/{dataset_name}'

P_SEQ = 6
Q_SEQ = 6

In [5]:
sorted(os.listdir('../dataprocess/seouldata'))

['floating-population-2017-a20.csv',
 'floating-population-2017-a30.csv',
 'floating-population-2017-a40.csv',
 'floating-population-2017-a50.csv',
 'floating-population-2017-a60.csv',
 'floating-population-2017-total.csv',
 'floating-population-2018-a20.csv',
 'floating-population-2018-a30.csv',
 'floating-population-2018-a40.csv',
 'floating-population-2018-a50.csv',
 'floating-population-2018-a60.csv',
 'floating-population-2018-total.csv',
 'local_people_area.geojson',
 'seoul_link_network.geojson',
 'speed-2017.csv',
 'speed-2018.csv',
 'subway-data-off-2017.csv',
 'subway-data-off-2018.csv',
 'subway-data-on-2017.csv',
 'subway-data-on-2018.csv',
 'volume_data_2017.csv',
 'volume_data_2018.csv',
 'volume_data_edit_2017.csv',
 'volume_data_edit_2018.csv',
 'vsensor_locations.geojson']

In [8]:
sub_on_df = pd.concat([
    pd.read_csv('../dataprocess/seouldata/subway-data-on-2017.csv', index_col=0),
    pd.read_csv('../dataprocess/seouldata/subway-data-on-2018.csv', index_col=0)
]).iloc[:-2]

sub_off_df = pd.concat([
    pd.read_csv('../dataprocess/seouldata/subway-data-off-2017.csv', index_col=0),
    pd.read_csv('../dataprocess/seouldata/subway-data-off-2018.csv', index_col=0)
]).iloc[:-2]

sub_on_df.index = pd.to_datetime(sub_on_df.index)
sub_off_df.index = pd.to_datetime(sub_off_df.index)

In [35]:
sub_on_df.shape

(17520, 275)

In [36]:
sub_off_df.shape

(17520, 275)

In [39]:
assert sum(sub_on_df.columns != sub_off_df.columns) == 0

In [10]:
assert len(sub_on_df) == len(sub_off_df) == 24*(365+365)

In [71]:
net_df = pd.read_json('../dataprocess/seouldata/station_coordinate.json')
net_gdf = gpd.GeoDataFrame(
    net_df, geometry=gpd.points_from_xy(x=net_df.lng, y=net_df.lat)
)
net_gdf.crs = 'EPSG:4326'

filtered_points = []
for i in sub_on_df.columns.astype(int):
    d = dict(net_gdf[net_gdf['code'] == i].iloc[0])
    assert len(d) != 0
    filtered_points.append(dict(network_gdf[network_gdf['code'] == i].iloc[0]))
    
network_gdf = gpd.GeoDataFrame(filtered_points)
network_gdf['code'] = network_gdf['code'].astype(int)

In [79]:
##################
data_columns = sub_on_df.columns
idset = set()
geo = []
for _, sensor in network_gdf.iterrows():
    id = f"{int(sensor['code'])}"
    lat = sensor.geometry.y
    lon = sensor.geometry.x
    if id not in idset:
        idset.add(id)
        geo.append([id, 'Point', '['+str(lon)+', '+str(lat)+']'])
        
geo = pd.DataFrame(geo, columns=['geo_id', 'type', 'coordinates'])
geo = geo[geo['geo_id'].isin(data_columns)]
geo.to_csv(dataname+'.geo', index=False)

In [80]:
import shapely
item_list = []
for _, item in geo.iterrows():
    itemdict = dict(item)
    itemdict['geometry'] = shapely.Point([float(v) for v in itemdict['coordinates'][1:-1].split(',')])    
    item_list.append(itemdict)
again_gdf = gpd.GeoDataFrame(item_list)
again_gdf.crs = 'epsg:4326'
again_5181_gdf = again_gdf.to_crs('epsg:5181')

In [85]:
#############################
dist_df = pd.read_csv('../dataprocess/seouldata/subway_distances.csv', index_col=0)
rel = []
reldict = dict()

for _, item in dist_df.iterrows():
    sid, eid, cost = str(item['from']), str(item['to']), item['cost']
    if sid not in idset or eid not in idset:
        continue
    if cost == 0:
        continue
    if (sid, eid) not in reldict:
        reldict[(sid, eid)] = len(reldict)
        rel.append([len(reldict) - 1, 'geo', sid, eid, cost])
            
rel = pd.DataFrame(rel, columns=['rel_id', 'type', 'origin_id', 'destination_id', 'cost'])
rel.to_csv(dataname+'.rel', index=False)

In [87]:
# data_df.index = pd.to_datetime(data_df.index)
# data_df.to_hdf(dataname + '_ALL.hdf', key='df')

In [88]:
# df = pd.read_hdf(dataname + '_ALL.hdf', key='df')
seniorlist = list(data_columns)
timeslot = pd.to_datetime(sub_on_df.index)


dataset1 = sub_on_df
dataset2 = sub_off_df

# dyna = []
dyna_id = 0
dyna_file = open(dataname+'.dyna', 'w')
dyna_file.write('dyna_id' + ',' + 'type' + ',' + 'time' + ',' + 'entity_id' + ',' + 'demand_on' + ',' + 'demand_off' + '\n')
for j in range(len(seniorlist)):
    entity_id = seniorlist[j]
    for i in range(timeslot.shape[0]):
        time = str(timeslot[i])+'Z'
        dyna_file.write(str(dyna_id) + ',' + 'state' + ',' + str(time)
                        + ',' + str(entity_id) + ',' + str(dataset1[seniorlist[j]][i])  + ',' + str(dataset2[seniorlist[j]][i]) + '\n')
        dyna_id = dyna_id + 1
        if dyna_id % 10000 == 0:
            print(str(dyna_id) + '/' + str(timeslot.shape[0]*len(seniorlist)))
dyna_file.close()
# dyna = pd.DataFrame(dyna, columns=['dyna_id', 'type', 'time', 'entity_id', 'traffic_speed'])
# dyna.to_csv(dataname+'.dyna', index=False)


config = dict()
config['geo'] = dict()
config['geo']['including_types'] = ['Point']
config['geo']['Point'] = {}
config['rel'] = dict()
config['rel']['including_types'] = ['geo']
config['rel']['geo'] = {'cost': 'num'}
config['dyna'] = dict()
config['dyna']['including_types'] = ['state']
config['dyna']['state'] = {'entity_id': 'geo_id', 'demand_on': 'num', 'demand_off': 'num'}
config['info'] = dict()
config['info']['data_col'] = ['demand_on', 'demand_off']
config['info']['weight_col'] = 'cost'
config['info']['data_files'] = [dataset_name]
config['info']['geo_file'] = dataset_name
config['info']['rel_file'] = dataset_name
config['info']['output_dim'] = 1
config['info']['time_intervals'] = 300
config['info']['init_weight_inf_or_zero'] = 'inf'
config['info']['set_weight_link_or_dist'] = 'dist'
config['info']['calculate_weight_adj'] = True
config['info']['weight_adj_epsilon'] = 0.1
json.dump(config, open(outputdir+'/config.json', 'w', encoding='utf-8'), ensure_ascii=False)

10000/4818000
20000/4818000
30000/4818000
40000/4818000
50000/4818000
60000/4818000
70000/4818000
80000/4818000
90000/4818000
100000/4818000
110000/4818000
120000/4818000
130000/4818000
140000/4818000
150000/4818000
160000/4818000
170000/4818000
180000/4818000
190000/4818000
200000/4818000
210000/4818000
220000/4818000
230000/4818000
240000/4818000
250000/4818000
260000/4818000
270000/4818000
280000/4818000
290000/4818000
300000/4818000
310000/4818000
320000/4818000
330000/4818000
340000/4818000
350000/4818000
360000/4818000
370000/4818000
380000/4818000
390000/4818000
400000/4818000
410000/4818000
420000/4818000
430000/4818000
440000/4818000
450000/4818000
460000/4818000
470000/4818000
480000/4818000
490000/4818000
500000/4818000
510000/4818000
520000/4818000
530000/4818000
540000/4818000
550000/4818000
560000/4818000
570000/4818000
580000/4818000
590000/4818000
600000/4818000
610000/4818000
620000/4818000
630000/4818000
640000/4818000
650000/4818000
660000/4818000
670000/4818000
6800